In [1]:
import requests
import time
import numpy as np

In [2]:
gm = np.load("puuids.npy")
m = np.load("masters.npy") #jusqu'a 10000
puuids = list(gm) + list(m)

In [3]:
key = 'RGAPI-0e2f42e5-53ed-4ca7-9551-c5651916ca25'

In [4]:
def get_puuid(puuids,list_player,key):
    c=1
    for p in list_player:
        sum_name = p['summonerName']
        try:
            req = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}'.format(sum_name,key))
            r = req.json()
            puuid = r['puuid']
            puuids.append(puuid)
        except Exception:
            pass
        time.sleep(0.9)
        print(len(puuids))
        #print(puuid)
        c+=1

In [5]:
def get_matches_id(m_ids,puuids,key):
    c=1
    for p in puuids:
        req = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?start=0&count=99&api_key={}'.format(p,key))
        r = req.json()
        try:
            for m in r:
                m_ids.append(m)
        except:
            print(r)
        time.sleep(0.9)
        print(len(m_ids))
        c+=1

In [6]:
GM_Match = np.load("grand_matches_id.npy")

M_Match = np.load("master_matches40000.npy") # Jusqu'a 400000 (avec doublon)

In [7]:
m_ids = list(GM_Match) + list(M_Match)
m_ids = list(set(m_ids))

In [8]:
len(m_ids)

319241

In [42]:
def add_match(match_id,api_key):
    Match = []
    req = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_id,api_key)
    time.sleep(0.9)
    r = requests.get(req)
    match_info = r.json()
    if match_info['info']['queueId']==420:
        Blue,Red = [],[]
        players = match_info['info']['participants']
        Winner = "Blue" if players[0]['win'] else "Red"
        for i in range(len(players)):
            if i<5:
                Blue.append(players[i]['individualPosition'])
                Blue.append(players[i]['championName'])
            else:
                Red.append(players[i]['individualPosition'])
                Red.append(players[i]['championName'])
        Match = [Blue,Red,Winner]
    return Match

In [43]:
def match(m_ids,key):
    Matchs = []
    for m in m_ids:
        ma = add_match(m,key)
        if ma!=[]:
            Matchs.append(ma)
        if len(Matchs)==1000:
            name = "data\\Matchs{}.npy".format(m_ids.index(m))
            Matchs = np.array(Matchs,dtype=list)
            np.save(name,Matchs)
            print("saved {}".format(name))
            Matchs = []
        if len(Matchs)%10==0:
            print(len(Matchs))

In [44]:
match(m_ids,key)

0
10
20
30
40
40
40
50
60
60
70
70
70
70
70
80
90
100
110
110
120
130
140
150
150
160
170
180
190
200
210
220
230
240
250
250
260
260
270
280
280
290
300
300
310
320
330
340
340
340
350
360
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
530
530
540
540
550
560
570
570
580
590
600
610
620
630
640
650
660
670
670
680
690
700
710
710
710
720
730
730
740
750
760
760
760
770
770
780
780
780
790
790
800
800
810
820
830
840
850
860
870
870
880
890
900
910
920
930
940
950
960
960
970
980
990
990
saved data\Matchs1426.npy
0
10
10
20
30
30
40
40
50
60
60
60
60
70
80
90
100
110
120
130
140
140
150
150
160
160
170
170
180
190
200
200
210
220
220
230
230
240
250
260
270
270
280
290
300
310
320
320
330
340
350
360
360
370
380
380
390
400
410
410
410
420
420
430
440
440
440
450
460
470
480
480
480
490
490
500
510
520
530
540
550
550
550
550
550
560
570
580
590
600
600
610
620
620
630
640
650
660
670
680
690
700
710
710
710
720
720
730
740
750
760
770
780
790
800
810
820
830
8

ConnectionError: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v5/matches/EUW1_6400984707?api_key=RGAPI-0e2f42e5-53ed-4ca7-9551-c5651916ca25 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016D8EEFDBE0>: Failed to establish a new connection: [WinError 10065] Une opération a été tentée sur un hôte impossible à atteindre'))

In [39]:
Matchs

[]